In [1]:
from dotenv import load_dotenv
import os

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI


# Warning control
import warnings
warnings.filterwarnings("ignore")

In [72]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')

In [15]:
def pdf_loader(file_path: str):
    loader = PyPDFLoader(file_path)
    text = ""
    for page in loader.load():
        text += page.page_content
    return text

In [62]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)

In [63]:
def split_boe_data_from_pdf(file):
    chunks_with_metadata = []
    

In [64]:
import requests
import xmltodict

url = "https://www.boe.es/datosabiertos/api/boe/sumario/20240101"

def get_items(departament):
        if type(departament["epigrafe"]) == dict:
            if type(departament["epigrafe"]["item"]) == list:
                for item in departament["epigrafe"]["item"]:
                    yield item
            else:
                yield departament["epigrafe"]["item"]
        else:
            for epigraf in departament["epigrafe"]:
                if type(epigraf["item"]) == list:
                    for item in epigraf["item"]:
                        yield item
                else:
                    yield epigraf["item"]


def get_metadata(url: str):
    headers = {"Accept": "application/xml"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data_dict = xmltodict.parse(response.content)
        chunks_with_metadata = []
        for seccion in data_dict["response"]["data"]["sumario"]["diario"]["seccion"]:
            for departament in seccion["departamento"]:
                for item in get_items(departament):
                    item_text = pdf_loader(file_path=f"C:/Users/2373225/projects/genai-1/data/pdfs/{item['identificador']}.pdf")
                    item_text_chunks = text_splitter.split_text(item_text)
                    chunk_seq_id = 0
                    for chunk in item_text_chunks:
                        chunks_with_metadata.append(
                            {
                                'text': chunk,
                                'boe_id': item["identificador"],
                                'chunkSeqId': chunk_seq_id,
                                'chunkId': f"{item['identificador']}-chunk{chunk_seq_id:04d}",
                                'source': item["url_pdf"]["#text"]
                            }
                        )
                        chunk_seq_id += 1
    return chunks_with_metadata


chunks_metadata = get_metadata(url=url)

In [65]:
# Instantiate Neo4j

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [66]:
chunks_metadata[0]

{'text': 'II. AUTORIDADES Y PERSONAL\nA. Nombramientos, situaciones e incidencias\nMINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL\n1 Resolución de 21 de diciembre de 2023, de la Subsecretaría, por la que se \nresuelve la convocatoria de libre designación, efectuada por Resolución de \n13 de noviembre de 2023.\nPor Resolución de la Subsecretaría de este Departamento, de 13 de noviembre \nde 2023 («Boletín Oficial del Estado» de 15 de noviembre de 2023), se anunció \nconvocatoria para cubrir, por el sistema de libre designación, puesto de trabajo en el \nMinisterio de Trabajo y Economía Social.\nDe conformidad con lo dispuesto en el artículo 20.1.c) de la Ley 30/1984, de 2 de \nagosto, de Medidas para la Reforma de la Función Pública, según la redacción dada al \nmismo por la Ley 23/1988, de 28 de julio, y en relación con el artículo 56 del Reglamento \naprobado por el Real Decreto 364/1995, de 10 de marzo, una vez acreditada la \nobservancia del proceso debido y el cumplimiento por parte de la

In [67]:
merge_chunk_node_query = """
MERGE(mergedChunk:Chunk {chunkId: $chunkParam.chunkId})
    ON CREATE SET 
        mergedChunk.text = $chunkParam.text,
        mergedChunk.boe_id = $chunkParam.boe_id,
        mergedChunk.source = $chunkParam.source, 
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId
        
RETURN mergedChunk
"""

kg.query(merge_chunk_node_query, params={'chunkParam':chunks_metadata[0]})

[{'mergedChunk': {'boe_id': 'BOE-A-2024-1',
   'text': 'II. AUTORIDADES Y PERSONAL\nA. Nombramientos, situaciones e incidencias\nMINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL\n1 Resolución de 21 de diciembre de 2023, de la Subsecretaría, por la que se \nresuelve la convocatoria de libre designación, efectuada por Resolución de \n13 de noviembre de 2023.\nPor Resolución de la Subsecretaría de este Departamento, de 13 de noviembre \nde 2023 («Boletín Oficial del Estado» de 15 de noviembre de 2023), se anunció \nconvocatoria para cubrir, por el sistema de libre designación, puesto de trabajo en el \nMinisterio de Trabajo y Economía Social.\nDe conformidad con lo dispuesto en el artículo 20.1.c) de la Ley 30/1984, de 2 de \nagosto, de Medidas para la Reforma de la Función Pública, según la redacción dada al \nmismo por la Ley 23/1988, de 28 de julio, y en relación con el artículo 56 del Reglamento \naprobado por el Real Decreto 364/1995, de 10 de marzo, una vez acreditada la \nobservancia del pr

In [68]:
kg.query("""
CREATE CONSTRAINT unique_chunk IF NOT EXISTS 
    FOR (c:Chunk) REQUIRE c.chunkId IS UNIQUE
""")

[]

In [69]:
kg.query("SHOW INDEXES")

[{'id': 0,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 11, 13, 16, 53, 13, 655000000, tzinfo=<UTC>),
  'readCount': 5},
 {'id': 1,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 2,
  'name': 'unique_chunk',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['chunkId'],
  'indexProvider': 'range-1.0',
  'owningConstraint': 'unique_chunk',
  'lastRead': neo4j.time.DateTime(2024, 11, 13, 16, 51, 56, 142000000, tzinfo=<UTC>),
  'rea

In [70]:
node_count = 0
for chunk in chunks_metadata:
    print(f"Creating `:Chunk` node for chunk ID {chunk['chunkId']}")
    kg.query(merge_chunk_node_query, 
            params={
                'chunkParam': chunk
            })
    node_count += 1
print(f"Created {node_count} nodes")

Creating `:Chunk` node for chunk ID BOE-A-2024-1-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-1-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-2-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-2-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-3-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-3-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-4-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-4-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-5-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-5-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-6-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-6-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-6-chunk0002
Creating `:Chunk` node for chunk ID BOE-A-2024-6-chunk0003
Creating `:Chunk` node for chunk ID BOE-A-2024-7-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-7-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-7-chunk00

## Create Vector Index

In [71]:
kg.query("""
         CREATE VECTOR INDEX `boe_chunks` IF NOT EXISTS
          FOR (c:Chunk) ON (c.textEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

[]

## Embed text of chunk nodes

In [ ]:
from time import sleep

# Total chunks to embed
total_nodes_to_embed = kg.query("""
    MATCH (chunk:Chunk)
    RETURN count(chunk) AS TotalChunksToEmbed
""")

# Function to get the number of nodes pending to embed
def pending_nodes_to_emded():
    number_embedded_nodes = kg.query(
      """
      MATCH (chunk:Chunk) WHERE chunk.textEmbedding IS NULL 
      RETURN count(chunk) AS NotEmbedChunks                          
      """
    )
    return number_embedded_nodes[0]["NotEmbedChunks"]

# Initialize pending_nodes variable
pending_nodes = pending_nodes_to_emded()

while pending_nodes != 0:
  try:
    print("Embedding nodes...")
    kg.query("""
        MATCH (chunk:Chunk WHERE chunk.textEmbedding IS NULL)
        WITH chunk
        LIMIT $embedding_batch_size
        WITH chunk, genai.vector.encode(
          chunk.text, 
          "AzureOpenAI", 
          {
            token: $openAiApiKey,
            resource: "knowledge-graphs",
            deployment: "text-embedding-ada-002"
          }) AS vector
        CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", vector)
        """, 
        params={"openAiApiKey": OPENAI_API_KEY, "embedding_batch_size": 50} 
        )
    
    pending_nodes = pending_nodes_to_emded()
    print(f"Number of pending nodes to embed: {pending_nodes}")
    sleep(60)
  except:
    print("Embedding has reach the limit rate per minut")
    sleep(60)
    continue

print("All nodes have been embeded!")

All nodes have been embeded


## Use similarity search to find relevant chunks

In [149]:
def neo4j_vector_search(question):
  """Search for similar nodes using the Neo4j vector index"""
  vector_search_query = """
    WITH genai.vector.encode(
      $question, 
      "AzureOpenAI", 
      {
        token: $openAiApiKey,
        resource: "knowledge-graphs",
        deployment: "text-embedding-ada-002"
      }) AS question_embedding
    CALL db.index.vector.queryNodes($index_name, $top_k, question_embedding) yield node, score
    RETURN score, node.text AS text
  """
  similar = kg.query(vector_search_query, 
                     params={
                      'question': question, 
                      'openAiApiKey':OPENAI_API_KEY,
                      'index_name': "boe_chunks", 
                      'top_k': 10})
  return similar

In [168]:
search_results = neo4j_vector_search(
    'BOE-2024-1'
)

In [169]:
search_results

[{'score': 0.9102630615234375,
  'text': 'cve: BOE-A-2024-10\nVerificable en https://www.boe.esANEXO II\nMinisterio de Defensa\n \nDATOS PERSONALES: \nPrimer apellido Segundo apellido: Nombre: \nDNI: Cuerpo o Escala: Grupo: NRP: \nDomicilio, calle y número: Localidad: Provincia: Teléfono: \n \nDESTINO ACTUAL: \nMinisterio: Centro Directivo: Localidad: \nPuesto: Nivel: Complemento Específico: Situación \n \nSOLICITA: Ser admitido a la convocatoria pública para proveer un puesto de trabajo por el sistema de libre designación, \nanunciado por Resolución                de fecha.............. («Boletín Oficial del Estado» de ............), para el puesto \nde trabajo siguiente: \n \nDesignación del Puesto de trabajo C. Específico Centro Directivo o Unidad de \nque dependa Localidad \n    \n \nSe adjunta currículum \n \n \n \n \n En               a     de                de 2.0     \n \n (firma del interesado)           \n \n \n \n \n \n \n \n \n \n \nSRA. SUBSECRETARIA DE DEFENSA (SUBDIRECCI

## Langchain RAG to chat with the KG

In [162]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

def get_llm():
    load_dotenv('.env', override=True)
    return AzureChatOpenAI(azure_deployment="gpt-4o-mini", api_version="2024-08-01-preview")

def get_embedding():
    load_dotenv('.env', override=True)
    return AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002", api_version="2023-05-15")

embedding_model = get_embedding()
llm_model = get_llm()

neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="boe_chunks",
    node_label="Chunk",
    text_node_properties=["text"],
    embedding_node_property="textEmbedding",
)

In [223]:
retriever = neo4j_vector_store.as_retriever(search_kwargs={'k': 15})

In [224]:
retriever.get_relevant_documents('Necesito informacion sobre: Resolución de 21 de diciembre de 2023, de la Subsecretaría, por la que se resuelve la convocatoria de libre designación, efectuada por Resolución de 13 de noviembre de 2023')

[Document(metadata={'boe_id': 'BOE-A-2024-4', 'source': 'https://www.boe.es/boe/dias/2024/01/01/pdfs/BOE-A-2024-4.pdf', 'chunkId': 'BOE-A-2024-4-chunk0000', 'chunkSeqId': 0}, page_content='\ntext: II. AUTORIDADES Y PERSONAL\nA. Nombramientos, situaciones e incidencias\nMINISTERIO DE TRABAJO Y ECONOMÍA SOCIAL\n1 Resolución de 21 de diciembre de 2023, de la Subsecretaría, por la que se \nresuelve la convocatoria de libre designación, efectuada por Resolución de \n13 de noviembre de 2023.\nPor Resolución de la Subsecretaría de este Departamento, de 13 de noviembre \nde 2023 («Boletín Oficial del Estado» de 15 de noviembre de 2023), se anunció \nconvocatoria para cubrir, por el sistema de libre designación, puesto de trabajo en el \nMinisterio de Trabajo y Economía Social.\nDe conformidad con lo dispuesto en el artículo 20.1.c) de la Ley 30/1984, de 2 de \nagosto, de Medidas para la Reforma de la Función Pública, según la redacción dada al \nmismo por la Ley 23/1988, de 28 de julio, y en r

In [225]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm_model, 
    chain_type="stuff", 
    retriever=retriever
)

In [226]:
def prettychain(question: str) -> str:
    """Pretty print the chain's response to a question"""
    response = chain({"question": question},
        return_only_outputs=True,)
    print(response['answer'])

In [229]:
prettychain('Explicame sobre: Resolución de 21 de diciembre de 2023, de la Universidad de Murcia, por la que se nombran Catedráticas y Catedráticos de Universidad.')

No tengo información sobre la "Resolución de 21 de diciembre de 2023, de la Universidad de Murcia, por la que se nombran Catedráticas y Catedráticos de Universidad". 


